In [1]:
from ds_class import Dstar
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import csv
from Custom_gym_env import CustomEnv
from threading import Thread

## Main

In [2]:
env = CustomEnv(obstacle_turn = True,
                vizualaze     = False, 
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.0,#0.005
                num_obs       = 5, 
                num_enemy     = 1, 
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 2000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])

/home/parkhomenko/PycharmProjects/Medvedev)learning/venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
direct_dict = {
                (0, 1): 6,
                (-1, 1): 5,
                (-1, 0): 4,
                (-1, -1): 3,
                (0, -1): 2,
                (1, -1): 1,
                (1, 0): 0,
                (1, 1): 7,
            }

### Фильтр для карты высот

In [4]:
def get_height_map(map):
    kernel = np.array([[1, 4, 6, 4, 1],
                       [4, 16, 24, 16, 4],
                       [6, 24, 36, 24, 6],
                       [4, 16, 24, 16, 4],
                       [1, 4, 6, 4, 1]])
    
    new_map = cv2.filter2D(map, -1, kernel)
    
    for i in range(7):
        new_map = cv2.filter2D(new_map, -1, kernel)/100
        new_map = np.clip(new_map, 0, 65000)
    
    return new_map

### Делаем папки 

In [5]:
# n_samples = 100000

# data_parts = {'train': n_samples * 0.75,
#              'test': n_samples * 0.1,
#              'val': n_samples * 0.15 }

# if not os.path.isdir('train'):
#     os.mkdir('train')

# if not os.path.isdir('test'):
#     os.mkdir('test')
    
# if not os.path.isdir('val'):
#     os.mkdir('val')

### Main

In [6]:

def data_generator(falder_name, count_samples):
    itr = 0
    if not os.path.isdir(falder_name):
        os.mkdir(falder_name)
    # env init
    env = CustomEnv(obstacle_turn = True,
                vizualaze     = False, 
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.0,#0.005
                num_obs       = 5, 
                num_enemy     = 1, 
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 2000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])

    with open(falder_name + '.csv', 'w') as csvfile:
        fieldnames = ['path', 'lable']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
        while itr < count_samples:    
            done = False
            state = env.reset()

            # img1 = state.img
            # img2 = state.img
            # img3 = state.img
            # Img_final = None
            
            while not done:
                heigh_map = get_height_map(state.img_D_star)
                d_goal = [int(state.target[0,0]),int(state.target[0,1])]
                d_start = [state.posRobot[0],state.posRobot[1]]
                
                d_star = Dstar(heigh_map, d_goal, d_start, heigh_map)
                d_star.Move_plan()
                d_star_path = d_star.query(d_start)
                
                act = direct_dict[tuple(d_star_path[1] - d_star_path[0])]
                # print(act)
                state, rew, done, info = env.step(act)
        ##########################################################
                path = './' + falder_name + '/' + str(itr) + '.png'
                itr+=1
                writer.writerow({'path': path, 'lable': str(act)})

                # img1 = img2
                # img2 = img3
                # img3 = state.img
                # new_img = cv2.addWeighted(img2, 0.4, img1, 0.2, 0)
                # Img_final = cv2.addWeighted(img3, 0.7, new_img, 0.5, 0)
                # resize
                # img = cv2.resize(Img_final, (65,65), interpolation = cv2.INTER_AREA)
                
                cv2.imwrite(path, state.img)

In [7]:
t1 = Thread(target=data_generator,args=('train', 75000))
t2 = Thread(target=data_generator,args=('test', 15000))
t3 = Thread(target=data_generator,args=('val', 10000))

In [8]:
t1.start()
t2.start()
t3.start()

In [9]:
t1.join()
t2.join()
t3.join()

### old loop

In [10]:
# n_samples = 100000

# data_parts = {'train': n_samples * 0.75,
#              'test': n_samples * 0.1,
#              'val': n_samples * 0.15 }

# if not os.path.isdir('train'):
#     os.mkdir('train')

# if not os.path.isdir('test'):
#     os.mkdir('test')
    
# if not os.path.isdir('val'):
#     os.mkdir('val')

In [11]:
# for part in data_parts.keys():
#     itr = 0

#     with open(part + '.csv', 'w') as csvfile:
#         fieldnames = ['path', 'lable']
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()
    
#         while itr < data_parts[part]:    
            
#             done = False
#             state = env.reset()
#             while not done:
#                 heigh_map = get_height_map(state.img_D_star)
#                 d_goal = [int(state.target[0,0]),int(state.target[0,1])]
#                 d_start = [state.posRobot[0],state.posRobot[1]]
                
#                 d_star = Dstar(heigh_map, d_goal, d_start, heigh_map)
#                 d_star.Move_plan()
#                 d_star_path = d_star.query(d_start)
                
#                 act = direct_dict[tuple(d_star_path[1] - d_star_path[0])]
#                 # print(act)
#                 state, rew, done, info = env.step(act)
#         ##########################################################
#                 path = './' + part + '/' + str(itr) + '.png'
#                 itr+=1
#                 writer.writerow({'path': path, 'lable': str(act)})
#                 cv2.imwrite(path, state.img)

In [12]:
# import csv
# with open('example4.csv', 'w') as csvfile:
#     fieldnames = ['first_name', 'last_name', 'Grade']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for i in range(10):
#         writer.writerow({'Grade': 'B', 'first_name': 'Alex', 'last_name': 'Brian'})

# print("Writing complete")